## In this notebook:

* on feature set 4

### Imports

In [1]:
#!pip install numpy 
#!pip install pandas
#!pip install torch
#!pip install sklearn

import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
import torch.optim as optim

### Manipulating data
* Import csv we want
* Map our features to usuable integers
* Map our labels to a one hot encoding

In [2]:
#################################################### 
df = pd.read_pickle("./pickled/chr2-4_featureset4.pkl")
df = df.replace({'-': 0})

# problematic isn't first:

#print("LENGTH",len(df.columns))
col = df.pop("Problematic")
df[col.name] = col

# Split our data into three separate sets for training purposes
train, test = train_test_split(df, shuffle=False)
train, valid = train_test_split(train, shuffle=False)

print(test.head())
print(train.shape, test.shape)

                       aapos codonpos  Ancestral_allele  Polyphen2_HVAR_pred  \
chr pos       ref alt                                                          
3   101244651 A   T      560        3                 2                    0   
    101244766 G   C      522        2                 2                    0   
    101269574 C   A       -1        0                 2                    0   
    101275694 G   C      212        2                 2                    0   
    101291499 A   C      171        3                 2                    0   

                          GenoCanyon_score HUVEC_fitCons_score  \
chr pos       ref alt                                            
3   101244651 A   T      0.995787957382137            0.567892   
    101244766 G   C      0.997773309808223            0.567892   
    101269574 C   A      0.998404006175971            0.057018   
    101275694 G   C    1.97468820369911E-4            0.620846   
    101291499 A   C      0.985239277427167 

### DataLoader
* Implement the Dataset class supplied by PyTorch
  * Used Map style
  * API: https://pytorch.org/docs/stable/data.html#map-style-datasets
* Instanciate DataLoader using Dataset instance

In [3]:
class DataSetdbNSFP(Dataset):
    def __init__(self, data):
        self.data = torch.FloatTensor(data.values.astype('float'))
        
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
            target = self.data[index][-1]
            data_val = self.data[index] [:-1]
            return data_val,target

def create_sampler(split_set):
    ratio = np.bincount(split_set.Problematic)
    classcount = ratio.tolist()
    weights = 1./torch.tensor(classcount, dtype=torch.float)
    sampleweights = weights[split_set.Problematic]
    sampler = WeightedRandomSampler(weights=sampleweights, num_samples=len(sampleweights))
    return sampler



train_dataset = DataSetdbNSFP(train)
valid_dataset = DataSetdbNSFP(valid)  
test_dataset = DataSetdbNSFP(test)

train_loader = DataLoader(train_dataset, batch_size=59, shuffle=False, sampler=create_sampler(train))
valid_loader = DataLoader(valid_dataset, batch_size=59, shuffle=False, sampler=create_sampler(valid))
test_loader = DataLoader(test_dataset, batch_size=59, shuffle=False, sampler=create_sampler(test))


### Model
* This model is a simple implementation
* Based on https://github.com/ieee8023/NeuralNetwork-Examples/blob/master/pytorch/pytorch-mnist.ipynb

In [4]:

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # In: nb of features, out: nb of examples
        self.fc = nn.Linear(14, 1000)
        self.fc2 = nn.Linear(1000, 1000)
        self.fc3 = nn.Linear(1000, 1000)
        self.fc4 = nn.Linear(1000, 1000)
         # In: nb of examples, out: nb of predictions
        self.fc5 = nn.Linear(1000, 2)

    def forward(self, x):
        x = x.view((-1, 14))
        h = F.relu(self.fc(x))
        h = self.fc2(h)
        h = F.relu(self.fc2(h))
        h = self.fc3(h)
        h = F.relu(self.fc3(h))
        h = self.fc4(h)
        h = F.relu(self.fc4(h))
        h = self.fc5(h)
        # Softmax to get the actual labels
        return F.softmax(h, dim=1)  ## Note: can potentially use this to get confidence of category predicted 
    
model = Model()
# if cuda:
#     model.cuda()

# Optimizer based on Adam algorithm, uses a slightly lower rate
optimizer = optim.Adam(model.parameters(), lr=1e-4)

### Training
* Run a fixed number of epochs
* After each epoch, run the validation set to adjust learning
* Print results
* Based on example: https://github.com/ieee8023/NeuralNetwork-Examples/blob/master/pytorch/pytorch-mnist.ipynb

In [5]:
EPOCHS = 20

model.train()
for epoch in range(EPOCHS):
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = Variable(data), Variable(target)
        print(data.shape)
        print(target.shape)
        
#         if cuda:
#             data, target = data.cuda(), target.cuda()
        
        optimizer.zero_grad()
        y_pred = model(data) 
        
        # Calculate and appy loss
        loss = F.cross_entropy(y_pred, target.long())
        loss.backward()
        optimizer.step()

        print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch+1,
            EPOCHS,
            batch_idx * len(data), 
            len(train_loader.dataset),
            100. * batch_idx / len(train_loader), 
            loss.cpu().data.item()))
    
    # NOTE: The Variable class is a wrapper class around a tensor with the added functionalities of back propagation
    # For x, take all cols of datafram except the last one
    evaluate_x = Variable(valid_loader.dataset.data[:, 0:-1].type_as(torch.FloatTensor()))
    print(evaluate_x)
    # For y, take last col
    evaluate_y = Variable(valid_loader.dataset.data[:, -1:])
#     if cuda:
#         evaluate_x, evaluate_y = evaluate_x.cuda(), evaluate_y.cuda()
    model.eval()
    output = model(evaluate_x)
    pred = output.data.max(1)[1]
    
    y_labels = torch.flatten(evaluate_y.data)
    print('Predictions:', pred)
    print('Actual values:', y_labels)
    d = pred.eq(y_labels).cpu()
    accuracy = d.sum().item()*1./d.size()[0]
    
    print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Validation Accuracy: {:.2f}%'.format(
        epoch+1,
        EPOCHS,
        len(train_loader.dataset), 
        len(train_loader.dataset),
        100. * batch_idx / len(train_loader), 
        loss.cpu().data.item(),
        accuracy*100))

torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [0/7773 (0%)]	Loss: 0.648285
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [59/7773 (1%)]	Loss: 0.618304
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [118/7773 (2%)]	Loss: 0.710403
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [177/7773 (2%)]	Loss: 0.745498
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [236/7773 (3%)]	Loss: 0.607015
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [295/7773 (4%)]	Loss: 0.602876
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [354/7773 (5%)]	Loss: 0.679524
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [413/7773 (5%)]	Loss: 0.639668
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [472/7773 (6%)]	Loss: 0.620546
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [531/7773 (7%)]	Loss: 0.765209
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [590/7773 (8%)]	Loss: 0.679875
torch.Size([59, 14])
torch.Size([59

 Train Epoch: 1/20 [5428/7773 (70%)]	Loss: 0.602534
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5487/7773 (70%)]	Loss: 0.548904
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5546/7773 (71%)]	Loss: 0.603322
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5605/7773 (72%)]	Loss: 0.513378
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5664/7773 (73%)]	Loss: 0.676029
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5723/7773 (73%)]	Loss: 0.622267
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5782/7773 (74%)]	Loss: 0.569030
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5841/7773 (75%)]	Loss: 0.721395
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5900/7773 (76%)]	Loss: 0.604910
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5959/7773 (77%)]	Loss: 0.641787
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [6018/7773 (77%)]	Loss: 0.697296
torch.Size([59, 14])
torch.Size([59])
 Train Epo

 Train Epoch: 2/20 [2773/7773 (36%)]	Loss: 0.621611
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [2832/7773 (36%)]	Loss: 0.616214
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [2891/7773 (37%)]	Loss: 0.520481
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [2950/7773 (38%)]	Loss: 0.673299
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [3009/7773 (39%)]	Loss: 0.664186
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [3068/7773 (39%)]	Loss: 0.529741
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [3127/7773 (40%)]	Loss: 0.577967
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [3186/7773 (41%)]	Loss: 0.602430
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [3245/7773 (42%)]	Loss: 0.603968
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [3304/7773 (42%)]	Loss: 0.580629
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [3363/7773 (43%)]	Loss: 0.544511
torch.Size([59, 14])
torch.Size([59])
 Train Epo

 Train Epoch: 3/20 [118/7773 (2%)]	Loss: 0.612115
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [177/7773 (2%)]	Loss: 0.601504
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [236/7773 (3%)]	Loss: 0.568396
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [295/7773 (4%)]	Loss: 0.590649
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [354/7773 (5%)]	Loss: 0.673543
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [413/7773 (5%)]	Loss: 0.518722
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [472/7773 (6%)]	Loss: 0.634791
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [531/7773 (7%)]	Loss: 0.575768
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [590/7773 (8%)]	Loss: 0.573174
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [649/7773 (8%)]	Loss: 0.612371
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [708/7773 (9%)]	Loss: 0.625075
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [767/7773 (10

 Train Epoch: 3/20 [5546/7773 (71%)]	Loss: 0.606111
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [5605/7773 (72%)]	Loss: 0.567392
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [5664/7773 (73%)]	Loss: 0.635732
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [5723/7773 (73%)]	Loss: 0.523905
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [5782/7773 (74%)]	Loss: 0.554737
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [5841/7773 (75%)]	Loss: 0.620583
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [5900/7773 (76%)]	Loss: 0.546167
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [5959/7773 (77%)]	Loss: 0.540604
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [6018/7773 (77%)]	Loss: 0.654923
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [6077/7773 (78%)]	Loss: 0.593524
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [6136/7773 (79%)]	Loss: 0.514621
torch.Size([59, 14])
torch.Size([59])
 Train Epo

...

 Train Epoch: 17/20 [5841/7773 (75%)]	Loss: 0.494329
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 17/20 [5900/7773 (76%)]	Loss: 0.511583
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 17/20 [5959/7773 (77%)]	Loss: 0.512590
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 17/20 [6018/7773 (77%)]	Loss: 0.565314
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 17/20 [6077/7773 (78%)]	Loss: 0.586787
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 17/20 [6136/7773 (79%)]	Loss: 0.515284
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 17/20 [6195/7773 (80%)]	Loss: 0.636910
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 17/20 [6254/7773 (80%)]	Loss: 0.664319
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 17/20 [6313/7773 (81%)]	Loss: 0.551352
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 17/20 [6372/7773 (82%)]	Loss: 0.610437
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 17/20 [6431/7773 (83%)]	Loss: 0.598960
torch.Size([59, 14])
torch.Size([59])

 Train Epoch: 18/20 [3127/7773 (40%)]	Loss: 0.565610
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 18/20 [3186/7773 (41%)]	Loss: 0.535781
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 18/20 [3245/7773 (42%)]	Loss: 0.564034
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 18/20 [3304/7773 (42%)]	Loss: 0.603935
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 18/20 [3363/7773 (43%)]	Loss: 0.578724
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 18/20 [3422/7773 (44%)]	Loss: 0.555711
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 18/20 [3481/7773 (45%)]	Loss: 0.596555
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 18/20 [3540/7773 (45%)]	Loss: 0.530712
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 18/20 [3599/7773 (46%)]	Loss: 0.530216
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 18/20 [3658/7773 (47%)]	Loss: 0.566995
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 18/20 [3717/7773 (48%)]	Loss: 0.569696
torch.Size([59, 14])
torch.Size([59])

 Train Epoch: 19/20 [354/7773 (5%)]	Loss: 0.558383
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [413/7773 (5%)]	Loss: 0.504862
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [472/7773 (6%)]	Loss: 0.569457
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [531/7773 (7%)]	Loss: 0.486104
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [590/7773 (8%)]	Loss: 0.633536
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [649/7773 (8%)]	Loss: 0.602453
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [708/7773 (9%)]	Loss: 0.611445
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [767/7773 (10%)]	Loss: 0.586981
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [826/7773 (11%)]	Loss: 0.610708
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [885/7773 (11%)]	Loss: 0.614358
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [944/7773 (12%)]	Loss: 0.634667
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/

 Train Epoch: 19/20 [5841/7773 (75%)]	Loss: 0.629263
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [5900/7773 (76%)]	Loss: 0.583563
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [5959/7773 (77%)]	Loss: 0.577815
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [6018/7773 (77%)]	Loss: 0.518230
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [6077/7773 (78%)]	Loss: 0.613211
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [6136/7773 (79%)]	Loss: 0.592912
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [6195/7773 (80%)]	Loss: 0.599858
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [6254/7773 (80%)]	Loss: 0.612101
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [6313/7773 (81%)]	Loss: 0.611943
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [6372/7773 (82%)]	Loss: 0.570439
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [6431/7773 (83%)]	Loss: 0.599523
torch.Size([59, 14])
torch.Size([59])

 Train Epoch: 20/20 [3127/7773 (40%)]	Loss: 0.619185
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 20/20 [3186/7773 (41%)]	Loss: 0.548034
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 20/20 [3245/7773 (42%)]	Loss: 0.536626
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 20/20 [3304/7773 (42%)]	Loss: 0.593929
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 20/20 [3363/7773 (43%)]	Loss: 0.522971
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 20/20 [3422/7773 (44%)]	Loss: 0.557009
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 20/20 [3481/7773 (45%)]	Loss: 0.524385
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 20/20 [3540/7773 (45%)]	Loss: 0.624463
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 20/20 [3599/7773 (46%)]	Loss: 0.557212
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 20/20 [3658/7773 (47%)]	Loss: 0.537674
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 20/20 [3717/7773 (48%)]	Loss: 0.603130
torch.Size([59, 14])
torch.Size([59])

### Testing
* Take testing dataset, run it against current model
* Evaluate testing accuracy

In [6]:
# Follow same steps as when we test with validation set, except with the test set
evaluate_x = Variable(test_loader.dataset.data[:, 0:-1].type_as(torch.FloatTensor()))
evaluate_y = Variable(test_loader.dataset.data[:, -1:])

model.eval()
output = model(evaluate_x)
pred = output.data.max(1)[1]


y_labels = torch.flatten(evaluate_y.data)
d = pred.eq(y_labels).cpu()
accuracy = d.sum().item()*1./d.size()[0]  ## Accuracy = (TP+TN)/(TP+TN+FP+FN)

print('\n Testing Accuracy: {:.2f}%'.format(

    accuracy*100))

print(len(pred))
print(len(y_labels))


 Testing Accuracy: 62.43%
3455
3455


### Metrics
* Use sklearn builtins to calculate different metrics

In [7]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
print("Confusion matrix: \n", confusion_matrix(y_labels, pred))

# Accuracy
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(y_labels, pred))

# Recall
from sklearn.metrics import recall_score
recall = recall_score(y_labels, pred, average=None)
print("Recall TP/(TP+FN):", recall)

# Precision
from sklearn.metrics import precision_score
precision = precision_score(y_labels, pred, average=None)
print("Precision TP/(TP+FP):",precision)


# F1 score
F1 = 2 * (precision * recall) / (precision + recall)
print("F1:", F1)

Confusion matrix: 
 [[ 740  338]
 [ 960 1417]]
Accuracy: 0.6243125904486252
Recall TP/(TP+FN): [0.6864564  0.59612958]
Precision TP/(TP+FP): [0.43529412 0.80740741]
F1: [0.53275738 0.68586641]


In [8]:
# Recalculating accuracy: TP + TN / (all exmaples)
print("accuracy", 2362/(1080+2362))
1080+2362

#Test shape, train shape
#(10325, 9) (3442, 9)

accuracy 0.6862289366647298


3442